In [18]:
from sklearn.tree import export_graphviz


In [19]:
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier


In [20]:
from matplotlib.colors import ListedColormap


In [21]:
import graphviz


In [22]:
from sklearn.preprocessing import StandardScaler
import pandas as pd
from mlxtend.feature_selection import SequentialFeatureSelector as SFS

In [23]:
f=pd.read_csv('Task4.csv')
f = f.set_index('Task')
df = f.to_numpy()
df.shape

(87, 32)

In [24]:
df

array([['c2001', 0, 1678.33, ..., 1752.350625, 347.07057142857127,
        0.0588235294117647],
       ['c2001', 0, 1404.52, ..., 1667.5464705882348, 337.62027027027017,
        0.0555555555555555],
       ['c2001', 0, 1491.17, ..., 1630.6761111111111, 311.5283720930231,
        0.1428571428571428],
       ...,
       ['kr007', 1, 1356.0, ..., 2333.0916666666667, 498.032, 0.0],
       ['kr007', 1, 1857.75, ..., 2163.9430769230767, 461.16206896551705,
        0.0714285714285714],
       ['kr007', 1, 2921.1, ..., 1355.102, 341.55720930232553,
        0.0476190476190476]], dtype=object)

In [25]:
X = df[:,2:]
y = df[:,1]
print(y)
y=y.astype('int')

#データチューニング
tuned_parameters = {'max_depth':[1, 2, 3, 4, 5],
         'min_samples_leaf':[1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
         'min_samples_split':[2, 3, 4, 5]}

#DecisionTreeClassifier(max_depth=4, min_samples_leaf=2, random_state=0)


from sklearn.model_selection import LeaveOneOut

# create loocv procedure
cv = LeaveOneOut()

clf = GridSearchCV(DecisionTreeClassifier(random_state=0),   # グリッドサーチで決定木を定義
                   tuned_parameters,
                   cv=cv,
                   n_jobs=-1,
                   verbose=2
                  )

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1]


In [26]:
stdsc = StandardScaler()
X_std = stdsc.fit_transform(X)

In [28]:
best_acc = 0
re = pd.DataFrame()
stdsc = StandardScaler()
X_std = stdsc.fit_transform(X)
for i in range(2,4):
    sffs = SFS(
    DecisionTreeClassifier(random_state=0), # 使う学習器
    k_features=i, #特徴をいくつまで選択するか
    forward=True, #Trueでforward selectionになる。Falseでback
    floating=True, #sffsを行うためのもの。
    verbose=1, #実行時のlogをどれだけ詳しく表示するか
    scoring='accuracy', # 評価指標
    cv=cv,#クロスバリデーション
    n_jobs=-1
    )

    sffs = sffs.fit(X_std, y)
    #選ばれた特徴
    X_train_sffs = X_std[:,list(sffs.k_feature_idx_)]

    X_train_sffs_std = stdsc.fit_transform(X_train_sffs)
    
    clf = GridSearchCV(DecisionTreeClassifier(random_state=0),   # グリッドサーチで決定木を定義
                   tuned_parameters,
                   cv=cv,
                   n_jobs=-1,
                   verbose=2
                  )
    
    clf.fit(X_train_sffs_std, y)

    print(clf.best_score_, clf.best_params_)
    
    if clf.best_score_ < best_acc - 0.05:
        break
    elif clf.best_score_ > best_acc:
        best_acc = clf.best_score_
    
    re_i = pd.DataFrame([[sffs.k_feature_idx_,clf.best_score_,clf.best_params_]], columns=['Feature','accuracy','param'])
    re = pd.concat([re,re_i])
    '''# トレーニングデータに対する精度
    pred_train = random_forest.predict(X_train_sffs)
    accuracy_train = accuracy_score(y_train, pred_train)

    # テストデータに対する精度
    pred_test = random_forest.predict(X_test_sffs)
    accuracy_test = accuracy_score(y_test, pred_test)

    re_i = pd.DataFrame([[sffs.k_feature_idx_,accuracy_train,accuracy_test]], columns=['Feature','TrainigScore','TestScore'])
    re = pd.concat([re,re_i])

re.to_csv('../result/RF_Task1.csv')'''

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:    0.7s finished
Features: 1/2[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  29 out of  29 | elapsed:    0.7s finished
Features: 2/2

Fitting 87 folds for each of 200 candidates, totalling 17400 fits
0.8735632183908046 {'max_depth': 3, 'min_samples_leaf': 4, 'min_samples_split': 2}


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:    0.7s finished
Features: 1/3[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  29 out of  29 | elapsed:    0.7s finished
Features: 2/3[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  28 out of  28 | elapsed:    0.8s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:    0.1s finished
Features: 3/3

Fitting 87 folds for each of 200 candidates, totalling 17400 fits
0.9195402298850575 {'max_depth': 4, 'min_samples_leaf': 4, 'min_samples_split': 2}


In [30]:
re.to_csv('DT_Task4_SFFS.csv')